In [1]:
import sqlite3
import pandas as pd

import pystac_client
import odc.stac

In [2]:
# Connecting to sqlite 
conn = sqlite3.connect('fire.db')
#conn = sqlite3.connect('fire.db')

# Create a cursor object
cursor = conn.cursor()

In [3]:
# set this to True if you run it the first time, to create the database
first = True

if first == True:
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv('./Data/base_data_1986-2018.csv')

    df.drop('time', axis=1, inplace=True)
    df.drop('index_right', axis=1, inplace=True)
    df.month = pd.to_datetime(df.month, format='%Y-%m')
    df = df[df['month'].dt.year != 1986]

    # Write the DataFrame to the SQLite database
    df.to_sql('ground_data', conn, if_exists='replace', index=False)

    # Commit the changes
    conn.commit()

In [4]:
cursor.execute('''SELECT * from ground_data LIMIT 10''')
cursor.fetchall()

[('1987-01-01 00:00:00',
  -39.05,
  146.34,
  'POINT (146.34 -39.05)',
  619.038818359375,
  6.10546875,
  288.10693359375,
  73.9599609375,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  0.0,
  5831.0),
 ('1987-01-01 00:00:00',
  -38.94,
  146.34,
  'POINT (146.34 -38.94)',
  624.3359375,
  5.265625,
  288.486083984375,
  72.8436279296875,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  4.0,
  5832.0),
 ('1987-01-01 00:00:00',
  -38.94,
  146.45,
  'POINT (146.45 -38.94)',
  612.244384765625,
  5.552734375,
  288.52587890625,
  72.5224609375,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  0.0,
  5942.0),
 ('1987-01-01 00:00:00',
  -38.83,
  145.9,
  'POINT (145.9 -38.83)',
  930.560791015625,
  6.072265625,
  288.916259765625,
  71.4388427734375,
  2.082829236984253,
  1.03,
  -0.75,
  0.0,
  0.0,
  5393.0),
 ('1987-01-01 00:00:00',
  -38.83,
  146.01,
  'POINT (146.01 -38.83)',
  937.9697265625,
  5.759765625,
  288.786865234375,
  71.83502197265625,
  2.082829236984253,
  1.03,
 

In [5]:
#Get all tables
def get_all_tables(cursor):
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor.execute(sql_query)
    return cursor.fetchall()

# Get column names and data types for each table
def get_table_info(cursor, table_name):
    cursor.execute(f'PRAGMA table_info({table_name})')
    return cursor.fetchall()

In [6]:
tables = [table[0] for table in get_all_tables(cursor)]

for table in tables:
    print(f"Table: {table}")
    for column in get_table_info(cursor, table):
        print(f"Column: {column[1]}, Type: {column[2]}")
    print("\n")

Table: ground_data
Column: month, Type: TIMESTAMP
Column: lat, Type: REAL
Column: lon, Type: REAL
Column: geometry, Type: TEXT
Column: mrso, Type: REAL
Column: sfcWind, Type: REAL
Column: tas, Type: REAL
Column: hurs, Type: REAL
Column: pdsi, Type: REAL
Column: MEI, Type: REAL
Column: SOI, Type: REAL
Column: bushfire_count, Type: REAL
Column: prescribed_count, Type: REAL
Column: grid_id, Type: REAL




In [7]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [8]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [9]:
def get_month_lat_long(cursor):
    sql_query = '''SELECT strftime('%Y-%m', month) as month, lat, lon, grid_id from ground_data'''
    cursor.execute(sql_query)
    return cursor.fetchall()

# def create_stac_data_table(cursor, conn):
#     # Create a new table with the required columns
#     cursor.execute('''
#     CREATE TABLE IF NOT EXISTS stac_data (
#         time TEXT,
#         y REAL,
#         x REAL,
#         grid_id REAL,
#         bs_pc_10 REAL,
#         bs_pc_50 REAL,
#         bs_pc_90 REAL,
#         pv_pc_10 REAL,
#         pv_pc_50 REAL,
#         pv_pc_90 REAL,
#         npv_pc_10 REAL,
#         npv_pc_50 REAL,
#         npv_pc_90 REAL
#     )
#     ''')

#     # Commit the changes
#     conn.commit()

# def write_stac_data_to_db(cursor, conn, data):
#     for item in data:
#         cursor.execute('''
#         INSERT INTO stac_data (time, y, x, grid_id, bs_pc_10, bs_pc_50, bs_pc_90, pv_pc_10, pv_pc_50, pv_pc_90, npv_pc_10, npv_pc_50, npv_pc_90)
#         VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
#         ''', (
#             item['time'], item['y'], item['x'], item['grid_id'],
#             item['bs_pc_10'], item['bs_pc_50'], item['bs_pc_90'],
#             item['pv_pc_10'], item['pv_pc_50'], item['pv_pc_90'],
#             item['npv_pc_10'], item['npv_pc_50'], item['npv_pc_90']
#         ))

#     # Commit the changes
#     conn.commit()

In [10]:
#create_stac_data_table(cursor, conn)

In [11]:
# data_tiles = get_month_lat_long(cursor)

# print(len(data_tiles))

# for data in data_tiles:

#     bbox = [data[2] - 0.05, data[1] - 0.05, data[2] + 0.05, data[1] + 0.05]

#     start_date = f"{data[0][:4]}-01-01"
#     end_date = f"{data[0][:4]}-12-31"

#     print(f"Searching for data in {start_date} to {end_date} for lat: {data[1]} and long: {data[2]}")

#     # Set product ID as the STAC "collection"
#     collections = ["ga_ls_fc_pc_cyear_3"]

#     # Build a query with the parameters above
#     query = catalog.search(
#         bbox=bbox,
#         collections=collections,
#         datetime=f"{start_date}/{end_date}",
#     )

#     if len(list(query.items())) == 0:
#         print("No data found")
#         continue

#     # Search the STAC catalog for all items matching the query
#     items = list(query.items())
#     print(f"Found: {len(items):d} datasets")

#     # Load the data using the odc.stac module
#     ds = odc.stac.load(
#                 items=items,
#                 crs="EPSG:3577",
#                 lat=(bbox[1], bbox[3]),
#                 lon=(bbox[0], bbox[2]),
#                 time=(start_date, end_date))
    
#     data_entry = {
#         'time': data[0],
#         'grid_id': data[3],
#         'bs_pc_10': float(ds.bs_pc_10.mean().values),
#         'bs_pc_50': float(ds.bs_pc_50.mean().values),
#         'bs_pc_90': float(ds.bs_pc_90.mean().values),
#         'pv_pc_10': float(ds.pv_pc_10.mean().values),
#         'pv_pc_50': float(ds.pv_pc_50.mean().values),
#         'pv_pc_90': float(ds.pv_pc_90.mean().values),
#         'npv_pc_10': float(ds.npv_pc_10.mean().values),
#         'npv_pc_50': float(ds.npv_pc_50.mean().values),
#         'npv_pc_90': float(ds.npv_pc_90.mean().values)
#     }

#     write_stac_data_to_db(cursor, conn, [data_entry])


In [12]:
# cursor.execute('''SELECT * FROM stac_data''')
# rows = cursor.fetchall()

# for row in rows:
#     print(row)

In [13]:
# Function to check if a column exists in a table
def column_exists(cursor, table_name, column_name):
	cursor.execute(f"PRAGMA table_info({table_name})")
	columns = [info[1] for info in cursor.fetchall()]
	return column_name in columns

# List of columns to add
columns_to_add = [
	'bs_pc_10 REAL DEFAULT NAN',
	'bs_pc_50 REAL DEFAULT NAN',
	'bs_pc_90 REAL DEFAULT NAN',
	'pv_pc_10 REAL DEFAULT NAN',
	'pv_pc_50 REAL DEFAULT NAN',
	'pv_pc_90 REAL DEFAULT NAN',
	'npv_pc_10 REAL DEFAULT NAN',
	'npv_pc_50 REAL DEFAULT NAN',
	'npv_pc_90 REAL DEFAULT NAN'
]

# Alter the table to add new columns if they don't already exist
for column in columns_to_add:
	column_name = column.split()[0]
	if not column_exists(cursor, 'ground_data', column_name):
		cursor.execute(f'ALTER TABLE ground_data ADD COLUMN {column}')

# Commit the changes
conn.commit()

In [14]:
def update_ground_data(cursor, conn, data):
    cursor.execute('''
    UPDATE ground_data
    SET bs_pc_10 = ?, bs_pc_50 = ?, bs_pc_90 = ?, pv_pc_10 = ?, pv_pc_50 = ?, pv_pc_90 = ?, npv_pc_10 = ?, npv_pc_50 = ?, npv_pc_90 = ?
    WHERE grid_id = ? AND month = ?
    ''', (
        data['bs_pc_10'], data['bs_pc_50'], data['bs_pc_90'],
        data['pv_pc_10'], data['pv_pc_50'], data['pv_pc_90'],
        data['npv_pc_10'], data['npv_pc_50'], data['npv_pc_90'],
        data['grid_id'], data['time']
    ))

    # Commit the changes
    conn.commit()

In [ ]:
data_tiles = get_month_lat_long(cursor)

print(len(data_tiles))

for data in data_tiles:

    bbox = [data[2] - 0.05, data[1] - 0.05, data[2] + 0.05, data[1] + 0.05]

    start_date = f"{data[0][:4]}-01-01"
    end_date = f"{data[0][:4]}-12-31"

    print(f"Searching for data in {start_date} to {end_date} for lat: {data[1]} and long: {data[2]}")

    # Set product ID as the STAC "collection"
    collections = ["ga_ls_fc_pc_cyear_3"]

    try:
        # Build a query with the parameters above
        query = catalog.search(
            bbox=bbox,
            collections=collections,
            datetime=f"{start_date}/{end_date}",
        )

        if len(list(query.items())) == 0:
            print("No data found")
            continue

    except Exception as e:
        
        print(f"An error occurred: {e}")
        continue

    # Search the STAC catalog for all items matching the query
    items = list(query.items())
    print(f"Found: {len(items):d} datasets")

    # Load the data using the odc.stac module
    ds = odc.stac.load(
                items=items,
                crs="EPSG:3577",
                lat=(bbox[1], bbox[3]),
                lon=(bbox[0], bbox[2]),
                time=(start_date, end_date))
    
    data_entry = {
        'time': data[0],
        'grid_id': data[3],
        'bs_pc_10': float(ds.bs_pc_10.mean().values),
        'bs_pc_50': float(ds.bs_pc_50.mean().values),
        'bs_pc_90': float(ds.bs_pc_90.mean().values),
        'pv_pc_10': float(ds.pv_pc_10.mean().values),
        'pv_pc_50': float(ds.pv_pc_50.mean().values),
        'pv_pc_90': float(ds.pv_pc_90.mean().values),
        'npv_pc_10': float(ds.npv_pc_10.mean().values),
        'npv_pc_50': float(ds.npv_pc_50.mean().values),
        'npv_pc_90': float(ds.npv_pc_90.mean().values)
    }

    update_ground_data(cursor, conn, data_entry)

187122
Searching for data in 1987-01-01 to 1987-12-31 for lat: -39.05 and long: 146.34
Found: 2 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.94 and long: 146.34
Found: 2 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.94 and long: 146.45
Found: 1 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.83 and long: 145.9
Found: 2 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.83 and long: 146.01
Found: 2 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.83 and long: 146.23
Found: 4 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.72 and long: 143.26
Found: 1 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.72 and long: 143.37
Found: 1 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.72 and long: 143.48
Found: 1 datasets
Searching for data in 1987-01-01 to 1987-12-31 for lat: -38.72 and long: 143.59
Found: 1 datasets
Searching for 

APIError: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>


In [17]:
cursor.execute('''SELECT * from ground_data''')
cursor.fetchall()

[('1987-01-01 00:00:00',
  -39.05,
  146.34,
  'POINT (146.34 -39.05)',
  619.038818359375,
  6.10546875,
  288.10693359375,
  73.9599609375,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  0.0,
  5831.0,
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN'),
 ('1987-01-01 00:00:00',
  -38.94,
  146.34,
  'POINT (146.34 -38.94)',
  624.3359375,
  5.265625,
  288.486083984375,
  72.8436279296875,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  4.0,
  5832.0,
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN'),
 ('1987-01-01 00:00:00',
  -38.94,
  146.45,
  'POINT (146.45 -38.94)',
  612.244384765625,
  5.552734375,
  288.52587890625,
  72.5224609375,
  0.4868292808532715,
  1.03,
  -0.75,
  0.0,
  0.0,
  5942.0,
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN',
  'NAN'),
 ('1987-01-01 00:00:00',
  -38.83,
  145.9,
  'POINT (145.9 -38.83)',
  930.560791015625,
  6.072265625,
  288.916259765625,
  71.43884277343

In [18]:
cursor.execute('SELECT COUNT(*) FROM ground_data')
count = cursor.fetchone()[0]
print(f"Number of entries in ground_data table: {count}")

Number of entries in ground_data table: 187122
